In [16]:
import sys
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional,BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.random import set_seed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tslearn.clustering import TimeSeriesKMeans
from sklearn.model_selection import train_test_split
import utils
from importlib import reload
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from datetime import datetime



In [18]:
print("HELLOAS")
feature_list = ['PctChgClCl','sumPctChgClCl_2','sumPctChgClCl_3','sumPctChgClCl_4','sumPctChgClCl_5','sumPctChgClCl_6',
                'PctChgVol','sumPctChgVol_2','sumPctChgVol_3','sumPctChgVol_4','sumPctChgVol_5','sumPctChgVol_6',
                'PctChgVix','sumPctChgVix_2','sumPctChgVix_3','sumPctChgVix_4','sumPctChgVix_5','sumPctChgVix_6','rsi','Close','Volume']
# feature_list = ['PctChgClCl','sumPctChgClCl_2','sumPctChgClCl_3','sumPctChgClCl_4','sumPctChgClCl_5','sumPctChgClCl_6','Close']
n_steps = 25

features = len(feature_list)
run_name = datetime.today().strftime('%Y-%m-%d_%H-%M')
run_number = 0
tstart = 2015
tend = 2020

# tickerList = ['aapl','spy','amd','qqq','amzn','meta','msft','f']
tickerList = ['aapl','spy','amd','qqq']
# tickerList = ['aapl']

trend_features = ['Close','Volume']
cluster_features = ['PctChgClCl','sumPctChgClCl_2','sumPctChgClCl_3','sumPctChgClCl_4','sumPctChgClCl_5','sumPctChgClCl_6']
print("HELLOAS")



HELLOAS
HELLOAS


In [ ]:
run_path = run_name+"/"+str(run_number)

if not os.path.exists(run_path):
    os.makedirs(run_path)

# Construct file name
file_name = os.path.join(run_path, "results.txt")

# Create and open the run_number.txt file for writing

run_info = ""
run_info += "Tickers: " + str(tickerList) + "\n"
run_info += "cluster_features: " + str(cluster_features) + "\n"
run_info += "n_steps: " + str(n_steps)+ "\n"
run_info += "Start: " + str(tstart) + " end: " + str(tend) + "\n\n"

with open(file_name, 'w') as file:
    file.write(run_info)

In [3]:

reload(utils)

# Initialize arrays as empty arrays with the necessary number of dimensions
X_train, X_test = np.empty((0, n_steps, len(feature_list))), np.empty((0, n_steps, len(feature_list)))
y_train, y_test = np.empty((0, 6)), np.empty((0, 6))

for ticker in tickerList: 
    df = utils.prepareStockDf(ticker,tstart)
    training_set, test_set = utils.train_test_split(df,tstart, tend, feature_list)

    dataset_total = df.loc[:,feature_list]
    test_set = dataset_total[len(dataset_total) - len(test_set) - n_steps :].values

    # scaler = MinMaxScaler(feature_range=(-1,1))
    # training_set = scaler.fit_transform(training_set)
    # test_set = scaler.transform(test_set)

    cur_X_train, cur_y_train = utils.split_sequence(training_set, n_steps)
    cur_X_test, cur_y_test = utils.split_sequence(test_set, n_steps)

    # Append cur_X_train, cur_X_test, cur_y_train, cur_y_test to the total arrays
    X_train = np.concatenate((X_train, cur_X_train), axis=0)
    X_test = np.concatenate((X_test, cur_X_test), axis=0)
    y_train = np.concatenate((y_train, cur_y_train), axis=0)
    y_test = np.concatenate((y_test, cur_y_test), axis=0)


display(X_train[0,:,-1])
display(y_train.shape)
display(X_test.shape)
display(y_test.shape)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


array([2.128184e+08, 2.571420e+08, 2.631884e+08, 1.604236e+08,
       2.374580e+08, 2.147980e+08, 1.986032e+08, 2.683676e+08,
       1.958264e+08, 2.400560e+08, 3.140532e+08, 1.995996e+08,
       1.943036e+08, 2.151856e+08, 1.858592e+08, 2.224600e+08,
       3.822748e+08, 5.859084e+08, 3.377456e+08, 3.349820e+08,
       2.509564e+08, 2.076628e+08, 2.805988e+08, 1.689848e+08,
       1.748264e+08])

(5924, 6)

(2696, 25, 21)

(2696, 6)

In [4]:

trend_feature_index = [feature_list.index(trend) for trend in trend_features]

X_train = utils.scale_patterns(X_train,trend_feature_index)
display(X_train[0:1,:,-2])

array([[0.23942673, 0.01863814, 0.01935514, 0.12616499, 0.42293904,
        0.43154138, 0.23369183, 0.30322595, 0.27311856, 0.05949832,
        0.        , 0.1956991 , 0.25519742, 0.45949832, 0.5010755 ,
        0.5096773 , 0.22580649, 0.66810012, 0.92544799, 0.800717  ,
        0.90609285, 0.90752685, 0.97275952, 1.        , 0.92759844]])

In [6]:
cluster_feature_index = [feature_list.index(cluster_feature) for cluster_feature in cluster_features]
X_cluster = utils.create_Cluster_Seq(X_train,cluster_feature_index)
display(X_cluster.shape)
display(X_cluster[0:1,:,-1])


(5924, 25, 6)

array([[-0.29968238, -0.54666594, -0.73175814, -0.57676085, -0.04383813,
         0.16766101,  0.00840907,  0.39843351,  0.35732939, -0.07598302,
        -0.5622445 , -0.30239192,  0.03736294,  0.21775969,  0.3121932 ,
         0.60907623,  0.32229079,  0.64626286,  0.89363605,  0.46572227,
         0.5434815 ,  0.53407611,  0.98339321,  0.42173664,  0.00535227]])

In [7]:
import math
n_clusters = math.ceil(math.sqrt(len(X_cluster))) // 3
# A good rule of thumb is choosing k as the square root of the number of points in the training data set in kNN

km = TimeSeriesKMeans(n_clusters=n_clusters)

labels = km.fit_predict(X_cluster)

In [8]:
reload(utils)
threshold_factor = .5 # Adjust as necessary

X_cluster_adj, X_train_adj, y_train_adj, labels_adj = utils.remove_outliers(X_cluster,X_train,y_train,labels,km,threshold_factor)
display(X_cluster.shape)
display(X_train.shape)
display(y_train.shape)
display(X_cluster_adj.shape)
display(X_train_adj.shape)
display(y_train_adj.shape)
display(labels_adj.shape)

(5924, 25, 6)

(5924, 25, 21)

(5924, 6)

(4963, 25, 6)

(4963, 25, 21)

(4963, 6)

(4963,)

In [ ]:
reload(utils)
utils.visualizeData(labels_adj,X_cluster_adj)


In [109]:
# cluster_assignments = km.predict(X_test)
# set(cluster_assignments)

In [11]:
import pickle 
visualDict = {'labels': labels, 'X_train':X_train}
pickle_name = os.path.join(run_path, "X_train.pkl")

with open(pickle_name, 'wb') as file:
    pickle.dump(visualDict,file)

In [12]:
if len(y_train.shape) == 1:
    df = pd.DataFrame({'label': labels_adj, 'target': y_train_adj})
else:
    # Convert the 2D y_train into a DataFrame
    y_df = pd.DataFrame(y_train_adj, columns=[f'target_{i}' for i in range(y_train_adj.shape[1])])
    df = pd.concat([pd.DataFrame({'label': labels_adj}), y_df], axis=1)

df.head()
display(df['label'].value_counts())


22    532
4     396
3     304
6     285
9     273
15    264
24    239
1     231
8     227
18    218
13    195
10    184
14    179
5     178
19    172
0     144
12    143
11    137
21    117
2     109
7     105
20     94
17     93
16     73
23     71
Name: label, dtype: int64

In [13]:
# display(X_train.shape)
# display(y_train.mean())
# display(cluster_X_train[0])

# label_filter = 1
# filtered_x_train, filtered_y_train = utils.filter_clusters(labels,label_filter,X_train_scaled,y_train_scaled)
# filtered_x_train.shape

In [14]:
def create_model():
    model_lstm = Sequential()

    # Using Bidirectional LSTM for the first layer
    model_lstm.add(Bidirectional(LSTM(units=100, activation='tanh', return_sequences=True), input_shape=(None, features)))
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(GRU(units=250, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(GRU(units=100, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=100, activation='tanh'))  # Changed number of units to 100
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(Dense(units=6))

    # Using Adam optimizer with a learning rate scheduler
    optimizer = Adam(learning_rate=0.001)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=1e-5)

    model_lstm.compile(optimizer=optimizer, loss="mse")
    return model_lstm







In [ ]:

for lab in set(labels_adj): 
    filtered_x_train, filtered_y_train = utils.filter_clusters(labels_adj, lab, X_train_adj, y_train_adj)

    X_test = utils.scale_patterns(X_test,trend_feature_index)
    # display(X_train[0:1,:,-1])
    X_cluster_test = utils.create_Cluster_Seq(X_test,cluster_feature_index)
    # display(X_cluster_test.shape)

    cluster_assignments = km.predict(X_cluster_test)

    # values, counts = np.unique(cluster_assignments, return_counts=True)

    # # Display the results
    # for value, count in zip(values, counts):
    #     print(f"Value: {value}, Count: {count}")

    X_cluster_test_adj,X_test_adj, y_test_adj, labels_test_adj = utils.remove_outliers(X_cluster_test,X_test,y_test,cluster_assignments,km,threshold_factor)

    filtered_x_test, filtered_y_test = utils.filter_clusters(labels_test_adj, lab, X_test_adj, y_test_adj)
    
    # display(X_cluster_test_adj.shape)
    # display(filtered_x_test.shape)
    # display(filtered_y_test.shape)

    if len(filtered_x_test) == 0:
        continue

    # display(filtered_x_train.shape)
    model_lstm = create_model()
    model_lstm.fit(filtered_x_train, filtered_y_train, batch_size=30, epochs=350, verbose=1)

    

    predicted_stock_price = model_lstm.predict(filtered_x_test)

    # Assuming predicted_stock_price has two columns
    results = pd.DataFrame({
        'predicted_1': predicted_stock_price[:, 0],
        'predicted_2': predicted_stock_price[:, 1],
        'predicted_3': predicted_stock_price[:, 2],
        'predicted_4': predicted_stock_price[:, 3],
        'predicted_5': predicted_stock_price[:, 4],
        'predicted_6': predicted_stock_price[:, 5],
        'real_1': filtered_y_test[:, 0],
        'real_2': filtered_y_test[:, 1],
        'real_3': filtered_y_test[:, 2],
        'real_4': filtered_y_test[:, 3],
        'real_5': filtered_y_test[:, 4],
        'real_6': filtered_y_test[:, 5]
    })

    # Assuming a correct prediction means both predicted features have the same sign as their corresponding real values
    results['same_1'] = ((results['predicted_1'] > 0) & (results['real_1'] > 0)) | ((results['predicted_1'] < 0) & (results['real_1'] < 0))
    results['same_2'] = ((results['predicted_2'] > 0) & (results['real_2'] > 0)) | ((results['predicted_2'] < 0) & (results['real_2'] < 0))
    results['same_3'] = ((results['predicted_3'] > 0) & (results['real_3'] > 0)) | ((results['predicted_3'] < 0) & (results['real_3'] < 0))
    results['same_4'] = ((results['predicted_4'] > 0) & (results['real_4'] > 0)) | ((results['predicted_4'] < 0) & (results['real_4'] < 0))
    results['same_5'] = ((results['predicted_5'] > 0) & (results['real_5'] > 0)) | ((results['predicted_5'] < 0) & (results['real_5'] < 0))
    results['same_6'] = ((results['predicted_6'] > 0) & (results['real_6'] > 0)) | ((results['predicted_6'] < 0) & (results['real_6'] < 0))
    accuracy1 = results['same_1'].sum() / len(results) * 100
    accuracy2 = results['same_2'].sum() / len(results) * 100
    accuracy3 = results['same_3'].sum() / len(results) * 100
    accuracy4 = results['same_4'].sum() / len(results) * 100
    accuracy5 = results['same_5'].sum() / len(results) * 100
    accuracy6 = results['same_6'].sum() / len(results) * 100

    # Construct the string
    output_string = (
        "Cluster Number: " + str(lab) +
        " \nAccuracy1D " + str(accuracy1) + " PredictedRet: " + str(results['predicted_1'].mean()) + " ActRet " + str(results['real_1'].mean() ) +
        " \nAccuracy2D " + str(accuracy2) + " PredictedRet: " + str(results['predicted_2'].mean()) + " ActRet " + str(results['real_2'].mean() ) +
        " \nAccuracy3D " + str(accuracy3) + " PredictedRet: " + str(results['predicted_3'].mean()) + " ActRet " + str(results['real_3'].mean() ) +
        " \nAccuracy4D " + str(accuracy4) + " PredictedRet: " + str(results['predicted_4'].mean()) + " ActRet " + str(results['real_4'].mean() ) +
        " \nAccuracy5D " + str(accuracy5) + " PredictedRet: " + str(results['predicted_5'].mean()) + " ActRet " + str(results['real_5'].mean() ) +
        " \nAccuracy6D " + str(accuracy6) + " PredictedRet: " + str(results['predicted_6'].mean()) + " ActRet " + str(results['real_6'].mean() ) +
        " Test set length: " + str(len(filtered_y_test)) + "\n"
    )

    # Write the string to a file
    with open(file_name, 'a') as f:
        f.write(output_string)




